In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import svm, datasets

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import xgboost
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error,r2_score

from sklearn.linear_model import BayesianRidge, LinearRegression

#from fbprophet import Prophet
#import keras
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#from keras.layers import Dropout

In [81]:
train_data = pd.read_csv("Train_Kaggle.csv")
test_data = pd.read_csv("Test_Kaggle.csv")
mapping = {k: v for v, k in enumerate(train_data.ProductCategory.unique())}
train_data['ProductCategory'] = train_data.ProductCategory.map(mapping)
test_data['ProductCategory'] = test_data.ProductCategory.map(mapping)
train_data = train_data.dropna(axis=0, subset=['Sales(In ThousandDollars)']) #remove row where if there is any 'NaN' value in column 'A'

#train_data.head(5)

In [82]:

X_train, X_test, y_train, y_test = train_test_split( train_data.iloc[:,0:3], train_data.iloc[:,3:4], test_size = 0.2, random_state = 100)

X_train = X_train.fillna(0)
y_train = y_train.fillna(0)
                               
X_test = X_test.fillna(0)
y_test = y_test.fillna(0)

np.count_nonzero(X_train.isnull().values)

0

In [83]:
#X_test

In [84]:
#X_train, X_test, y_train, y_test = train_test_split( train_data.iloc[:,0:3], train_data.iloc[:,3:4], test_size = 0.2, random_state = 100)
X_train = train_data.iloc[:,0:3]
y_train = train_data.iloc[:,3:4]
X_test = test_data.iloc[:,0:3]
#y_test = test_data.iloc[:,3:4]

#columns = ['Policing_Area', 'Weekday_of_Collision', 'Hour_of_Collision', 'Day_of_Collision', 'Month_of_Collision', 'Junction_Detail', 'Carriageway_Type', 'Junction_Control' ,'Special_Conditions_at_Site', 'Ped_Crossing_HC', 'Ped_Crossing_PC']
#X_train.drop(columns, inplace=True, axis=1)
#X_test.drop(columns, inplace=True, axis=1)

X_train = X_train.fillna(0)
y_train = y_train.fillna(0)
                               
X_test = X_test.fillna(0)
y_test = y_test.fillna(0)

np.count_nonzero(X_train.isnull().values)

0

In [85]:
y_train.shape

(204, 1)

In [86]:
model = RandomForestRegressor(100, oob_score = 1,n_jobs = 1,random_state =1)
#model = RandomForestRegressor(100, oob_score = 1,n_jobs = 1,random_state =42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#y_pred
#print(model.score(X_test, y_test))
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
rmse_value = rmse(y_pred,y_test.values)
rmse_value

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


1984.1292761451389

In [104]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = xgboost.XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X_train, y_train)
print(grid.best_params_)
y_pred = grid.best_estimator_.predict(X_test)
y_pred

# Print the r2 score
#print(r2_score(y_test, y_pred))

{'colsample_bytree': 1.0, 'gamma': 0.3, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.6}


array([3030.3538 ,  557.0536 , 1126.2167 , 3405.296  ,  591.9918 ,
       1161.1553 , 4130.026  ,  730.5864 , 1266.5648 , 4232.4756 ,
        770.1901 , 1251.1101 , 4304.3657 ,  778.2045 , 1267.8097 ,
       3978.1284 ,  710.94653, 1222.9647 , 3900.9329 ,  651.33795,
       1163.9568 , 4146.145  ,  712.9546 , 1223.7563 , 4338.039  ,
        729.5168 , 1196.0896 , 4353.1196 ,  770.043  , 1234.2863 ,
       4526.197  ,  854.4117 , 1396.1567 , 5372.8237 , 1269.6743 ,
       1773.0586 ], dtype=float32)

In [105]:
#print(model.score(X_test, y_test))
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
rmse_value = rmse(y_pred,y_test.values)
rmse_value

1988.5927401706663

In [89]:
#df = pd.DataFrame(columns = ["Year","Sales(In ThousandDollars)"])
#df["Year"] = X_test["Year"].values
df = pd.DataFrame(columns = ["Sales(In ThousandDollars)"])
df["Sales(In ThousandDollars)"] = y_pred
df.index = np.arange(1, len(df)+1)
df.index.name = "Year"
#df.to_csv('team1_submission_RFR_v1.csv',index=False)
df.to_csv('submission_xgb_cv_v2.csv',index=True)
df

,Sales(In ThousandDollars)
Year,
1,3030.353760
2,557.053589
3,1126.216675
4,3405.295898
5,591.991821
6,1161.155273
7,4130.025879
8,730.586426
9,1266.564819
